In [ ]:
import echopype as ep
import xarray as xr

import panel
import pandas
import hvplot.xarray  # noqa
import holoviews as hv

In [2]:
MVBS_ds = xr.open_mfdataset(
    paths = 'concatenated_MVBS.nc', 
    data_vars = 'minimal', coords='minimal',
    combine = 'by_coords'
)
MVBS_ds

<xarray.Dataset>
Dimensions:            (channel: 4, echo_range: 150, ping_time: 875)
Coordinates:
  * channel            (channel) object 'GPT  18 kHz 009072058c8d 1-1 ES18-11...
  * echo_range         (echo_range) float64 0.0 5.0 10.0 ... 735.0 740.0 745.0
  * ping_time          (ping_time) datetime64[ns] 2017-07-24T19:30:00 ... 201...
    time1              (ping_time) datetime64[ns] dask.array<chunksize=(875,), meta=np.ndarray>
Data variables:
    Sv                 (channel, ping_time, echo_range) float64 dask.array<chunksize=(4, 875, 150), meta=np.ndarray>
    frequency_nominal  (channel) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude          (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
    latitude           (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.7.1
    processing_time:              2023-05-30T17:40:45Z
    processing_function:          commongrid.compute_MVBS

In [3]:
MVBS_ds["Sv"].hvplot.image(
    x = 'ping_time', 
    y = 'echo_range', 
    color = 'Sv', 
    cmap = 'jet',
    xlabel = 'Time (UTC)',
    ylabel = 'Depth (m)',
    clabel = 'Sv(dB)'
).options(width = 800)

BokehModel(combine_events=True, render_bundle={'docs_json': {'cb3540ad-bf64-4a7f-8fea-93b6ae0fe19e': {'version…

In [4]:
from echogram import Echogram
from echostats import EchoStats, simple_hist

In [5]:
gram = MVBS_ds["Sv"].sel(channel = 'GPT  18 kHz 009072058c8d 1-1 ES18-11').hvplot.image(
    x = 'ping_time', 
    y = 'echo_range', 
    color = 'Sv', 
    cmap = 'jet',
    clim=(-90, -30),
    xlabel = 'Time (UTC)',
    ylabel = 'Depth (m)',
    clabel = 'Sv(dB)'
).options(width = 800)

simple_hist(gram)

BokehModel(combine_events=True, render_bundle={'docs_json': {'415344e4-a833-49b7-a7f0-55efde340f1f': {'version…

In [6]:
MVBS_ds.Sv.interactive.sel(channel = panel.widgets.Select).hvplot(
        kind = 'image',
        x = 'ping_time', 
        y = 'echo_range', 
        color = 'Sv', 
        cmap = 'jet',
        clim=(-90, -30),
        xlabel = 'Time (UTC)',
        ylabel = 'Depth (m)',
        clabel = 'Sv(dB)'
        ).options(width=800)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6de253ce-b56a-4deb-a12e-d3ee7e41b087': {'version…

In [7]:
channel_select = panel.widgets.Select(
            name = 'Channel Select',
            options = MVBS_ds.channel.values.tolist())

In [8]:
lower_time = pandas.to_datetime(MVBS_ds.ping_time.data[0])

upper_time = pandas.to_datetime(MVBS_ds.ping_time.data[-1])

datetime_range_input = panel.widgets.DatetimeRangeInput(
            name = 'Datetime Range Input',
            start = lower_time,
            end = upper_time,
            value = (lower_time, upper_time))

In [9]:
def create_echogram(channel_select, datetime_range_input):
        
    lower_time = datetime_range_input[0]
    
    upper_time = datetime_range_input[-1]

    image = MVBS_ds.Sv.sel(
        channel = channel_select, 
        ping_time=slice(lower_time,upper_time)).hvplot(
            kind = 'image',
            x='ping_time', 
            y='echo_range', 
            color='Sv', 
            cmap='jet',
            clim=(-90, -30),
            xlabel='Time (UTC)',
            ylabel='Depth (m)'
            ).options(invert_yaxis=False)
    
    return image

In [10]:
bind_gram = panel.bind(create_echogram, channel_select = channel_select, datetime_range_input = datetime_range_input)

In [11]:
panel.Column(channel_select,datetime_range_input, bind_gram)

BokehModel(combine_events=True, render_bundle={'docs_json': {'35949273-487c-4972-b1be-77f7c6a52c61': {'version…

In [12]:
echogram = Echogram(MVBS_ds)


In [13]:
panel.Row(echogram.widgets, echogram.view_gram) 

BokehModel(combine_events=True, render_bundle={'docs_json': {'55dae2fb-9d11-482e-8e0b-818b3a6cc2a9': {'version…

In [14]:
echogram.get_box_data()


<xarray.Dataset>
Dimensions:            (echo_range: 150, ping_time: 875)
Coordinates:
    channel            <U36 'GPT  18 kHz 009072058c8d 1-1 ES18-11'
  * echo_range         (echo_range) float64 0.0 5.0 10.0 ... 735.0 740.0 745.0
  * ping_time          (ping_time) datetime64[ns] 2017-07-24T19:30:00 ... 201...
    time1              (ping_time) datetime64[ns] dask.array<chunksize=(875,), meta=np.ndarray>
Data variables:
    Sv                 (ping_time, echo_range) float64 dask.array<chunksize=(875, 150), meta=np.ndarray>
    frequency_nominal  float64 dask.array<chunksize=(), meta=np.ndarray>
    longitude          (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
    latitude           (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.7.1
    processing_time:              2023-05-30T17:40:45Z
    processing_function:          commongrid.compute_MVBS

In [15]:
panel.Row(echogram.widgets, echogram.view_all_gram) 

BokehModel(combine_events=True, render_bundle={'docs_json': {'c7f2a18f-348d-48a2-a37f-43c0daa9bad6': {'version…

In [16]:
echostats = EchoStats(MVBS_ds)


In [17]:
panel.Row(echostats.widgets , panel.Column(echostats.view_gram, echostats.view_table))

BokehModel(combine_events=True, render_bundle={'docs_json': {'f83e2c20-86da-4cdd-9fae-5398e5383f52': {'version…

In [18]:
panel.Row(echostats.widgets , panel.Column(echostats.view_gram, echostats.view_hist))


BokehModel(combine_events=True, render_bundle={'docs_json': {'92bb5a96-362d-474e-ba03-7edc50ce60b8': {'version…

In [19]:
panel.Row(echostats.widgets , panel.Column(echostats.view_all_gram, echostats.view_all_table))


BokehModel(combine_events=True, render_bundle={'docs_json': {'fa85820e-c2a3-483f-9673-b601af9f2c24': {'version…

In [20]:
panel.Row(echostats.widgets , panel.Column(echostats.view_all_gram, echostats.view_overlay_hist))


BokehModel(combine_events=True, render_bundle={'docs_json': {'5cbbe84d-f909-4f0b-b022-69f73d07a110': {'version…

In [21]:
panel.Row(echostats.widgets , panel.Column(echostats.view_all_gram, echostats.view_layout_hist))

BokehModel(combine_events=True, render_bundle={'docs_json': {'fb7d74d9-73a5-4a1b-bb00-46a1c6820f9a': {'version…

In [22]:
from echomap import EchoMap
panel.extension("vtk")

echomap = EchoMap(MVBS_ds)


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


In [23]:

panel.Row(echomap.widgets, panel.Column(echomap.view_gram, echomap.view_box_map, echomap.view_curtain))

/Users/dingruilei/Desktop/echoshader/echomap.py:57: GeoviewsDeprecationWarning: 'Wikipedia' is deprecated and will be removed in version 1.11, use 'OSM' instead.
  tiles = getattr(geoviews.tile_sources, map_tiles)


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


BokehModel(combine_events=True, render_bundle={'docs_json': {'83b185f0-ba69-4ddf-ad0a-a9bc7b690eea': {'version…

Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Callback.process_on_change() done, defined at /Users/dingruilei/miniconda3/envs/echoshader-dev/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p5186', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/dingruilei/miniconda3/envs/echoshader-dev/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/dingruilei/miniconda3/envs/echoshader-dev/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/dingruilei/miniconda3/envs/echoshader-dev/lib/python3.10/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bo

AttributeError: 'str' object has no attribute 'param'